<a href="https://colab.research.google.com/github/bluebynick/CEE-9730-Data-Science-for-Geotechnical-Engineers/blob/Assignments/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!apt-get install -y openjdk-11-jdk
!pip install tabula-py

import tabula as tb
import pandas as pd

#qu 1
RM_dataFrame = tb.read_pdf("raw data (002).pdf", pages ="all");
df = RM_dataFrame[0]
df.dropna(axis=0, how ="any"); #from tut 3
df.fillna(0); # there's so many 0's in the data that i feel like
# filling the gaps with 0 is more reasonable than the median, mode, etc.
print(df);

#qu 2a
import seaborn as sns #this is what the internet said to use
import matplotlib.pyplot as plt

#generate the pairplot, first by grabbing numeric columns then sending it to .pairplot()
numeric_cols = df.select_dtypes(include='number').columns.tolist()

#find which in the df is correlated w C_mpa
correlation = df[numeric_cols].corr()['C_mpa'].abs().sort_values(ascending=False)

#grab the top ones and set them for the pairplot
topFeatures = correlation.index[1:5].tolist()
featuresToPlot = topFeatures + ['C_mpa']

#the pairplot
sns.pairplot(df[featuresToPlot], diag_kind='kde', plot_kws={'alpha':0.6})
plt.show()

#use this .corr() thing to look through my data and generate a heatmap to illustrate
sns.heatmap(df[featuresToPlot].corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.show()

#sanity check
#print(df.select_dtypes(include='number').corr()['C_mpa'].sort_values(ascending=False))

#qu2b.
from sklearn.cluster import KMeans
import numpy as np

n_cluster = 10
clusterer = KMeans(n_clusters=n_cluster)
X = df.to_numpy()
#cluster_labels = clusterer.fit_predict(X)
plt.show()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.28+6-1ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
ERROR: Operation cancelled by user


KeyboardInterrupt: 